<a href="https://colab.research.google.com/github/arnisafazla/Neural_Networks_Implementations/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from numpy.random import seed
from numpy.random import randn
from numpy import linspace

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = h5py.File("/content/drive/MyDrive/EEE443/hw3/assign3_data3.h5", "r")
X_train = np.array(file['trX'])
X_test = np.array(file['tstX'])
Y_train = np.array(file['trY'])
Y_test = np.array(file['tstY'])

index = [*range(X_train.shape[0])]
random.shuffle(index)
X_train = np.array([X_train[i] for i in index])
Y_train = np.array([Y_train[i] for i in index])

index = [*range(X_test.shape[0])]
random.shuffle(index)
X_test = np.array([X_test[i] for i in index])
Y_test = np.array([Y_test[i] for i in index])

n_val_x = int(X_train.shape[0] * 0.1)
X_val = X_train[0:n_val_x]
X_train = X_train[n_val_x:]
Y_val = Y_train[0:n_val_x]
Y_train = Y_train[n_val_x:]

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, X_val.shape, Y_val.shape)

(2700, 150, 3) (600, 150, 3) (2700, 6) (600, 6) (300, 150, 3) (300, 6)


In [ ]:
learning_rate = 0.1
momentum = 0.85
batch_size = 32                  
in_shape = (150,3)
n_classes = 6
n_units = 128
n_hidden1 = 100
n_hidden2 = 50
# other hidden layers here
max_epochs = 20
# extra hyperparameters
min_clip_value = -10
max_clip_value = 10
bptt_truncate = 5

In [ ]:
def categorical_cross_entropy(W_in, W_rec, W_hid1, W_hid2, W_out, b_in, b_rec, b_hid1, b_hid2, b_out, X, Y):
  c_matrix = np.zeros((n_classes, n_classes))
  N = Y.shape[0]
  prev_v = np.zeros((N, n_units))
  for t in range(in_shape[0]):
    new_input = X[:,t,:]  # N,3
    # print(new_input.shape)
    mul_in = np.dot(new_input, W_in.T) + b_in
    # print(mul_in.shape)
    mul_rec = np.dot(prev_v, W_rec.T) + b_rec
    add = mul_rec + mul_in
    v = tanh(add)
    mul_hid1 = np.dot(v, W_hid1.T) + b_hid1
    out_hid1 = tanh(mul_hid1)
    mul_hid2 = np.dot(mul_hid1, W_hid2.T) + b_hid2
    out_hid2 = tanh(mul_hid2)
    mul_out = np.dot(out_hid2, W_out.T) + b_out
    pred = softmax(mul_out)
    prev_v = v       

  loss = - np.sum(Y * np.log2(pred)) / N
  for sample in range(pred.shape[0]):
    c_matrix[np.argmax(pred[sample]), np.argmax(Y[sample])] += 1
  return loss, c_matrix
# SOFTMAX ACTIVATION FUNCTION
def softmax(X):
  x_max = np.array([np.max(row) for row in X])
  sum = np.array([np.sum(np.exp(row - np.max(row))) for row in X])
  prob = np.array([np.exp(X[i] - np.max(X[i])) / sum[i] for i in range(X.shape[0])])
  return prob


In [ ]:
categorical_cross_entropy(model.W_in, model.W_rec, model.W_hid1, model.W_out, model.b_in, model.b_rec, model.b_hid1, model.b_out, X_val, Y_val)

In [ ]:
def tanh(x):
  return np.tanh(x)

def tanh_derivative(x):
  return 1.0 - np.tanh(x)**2

def xavier(n_samples):
  lower, upper = -(1.0 / np.sqrt(n_samples)), (1.0 / np.sqrt(n_samples))
  numbers = np.random.rand(n_samples)
  return lower + numbers * (upper - lower)

def plot_history(train, val):
  plt.subplot(2, 1, 1)
  plt.plot(train, label='train_loss')
  plt.plot(val, label='val_loss')
  plt.legend()
  return plt.figure

In [ ]:
def clip(dW_in, dW_rec, dW_hid1, dW_hid2, dW_out, db_in, db_rec, db_hid1, db_hid2, db_out):
  # clip the values
  if dW_in.max() > max_clip_value:
    dW_in[dW_in > max_clip_value] = max_clip_value
  if dW_out.max() > max_clip_value:
    dW_out[dW_out > max_clip_value] = max_clip_value
  if dW_rec.max() > max_clip_value:
    dW_rec[dW_rec > max_clip_value] = max_clip_value
  if dW_hid1.max() > max_clip_value:
    dW_hid1[dW_hid1 > max_clip_value] = max_clip_value
  if dW_hid2.max() > max_clip_value:
    dW_hid2[dW_hid2 > max_clip_value] = max_clip_value
      
  
  if dW_in.min() < min_clip_value:
    dW_in[dW_in < min_clip_value] = min_clip_value
  if dW_out.min() < min_clip_value:
    dW_out[dW_out < min_clip_value] = min_clip_value
  if dW_rec.min() < min_clip_value:
    dW_rec[dW_rec < min_clip_value] = min_clip_value
  if dW_hid1.min() < min_clip_value:
    dW_hid1[dW_hid1 < min_clip_value] = min_clip_value
  if dW_hid2.min() < min_clip_value:
    dW_hid2[dW_hid2 < min_clip_value] = min_clip_value

  if db_in.max() > max_clip_value:
    db_in[db_in > max_clip_value] = max_clip_value
  if db_out.max() > max_clip_value:
    db_out[db_out > max_clip_value] = max_clip_value
  if db_rec.max() > max_clip_value:
    db_rec[db_rec > max_clip_value] = max_clip_value
  if db_hid1.max() > max_clip_value:
    db_hid1[db_hid1 > max_clip_value] = max_clip_value
  if db_hid2.max() > max_clip_value:
    db_hid2[db_hid2 > max_clip_value] = max_clip_value
  
  if db_in.min() < min_clip_value:
    db_in[db_in < min_clip_value] = min_clip_value
  if db_out.min() < min_clip_value:
    db_out[db_out < min_clip_value] = min_clip_value
  if db_rec.min() < min_clip_value:
    db_rec[db_rec < min_clip_value] = min_clip_value
  if db_hid1.min() < min_clip_value:
    db_hid1[db_hid1 < min_clip_value] = min_clip_value
  if db_hid2.min() < min_clip_value:
    db_hid2[db_hid2 < min_clip_value] = min_clip_value

  return dW_in, dW_rec, dW_hid1, dW_hid2, dW_out, db_in, db_rec, db_hid1, db_hid2, db_out

In [ ]:
class RNN(object):
  def __init__(self):
    self.W_in = xavier(n_units * in_shape[1]).reshape(n_units, in_shape[1])   # input to the layer
    self.b_in = xavier(n_units)
    self.rW_in = np.zeros(self.W_in.shape)
    self.rb_in = np.zeros(self.b_in.shape)
    self.W_rec = xavier(n_units * n_units).reshape(n_units, n_units)  # input from the past
    self.b_rec = xavier(n_units)
    self.rW_rec = np.zeros(self.W_rec.shape)
    self.rb_rec = np.zeros(self.b_rec.shape)
    # probably a hidden dense layer here
    self.W_hid1 = xavier(n_hidden1 * n_units).reshape(n_hidden1, n_units) # first hidden layer of MLP
    self.b_hid1 = xavier(n_hidden1)
    self.rW_hid1 = np.zeros(self.W_hid1.shape)
    self.rb_hid1 = np.zeros(self.b_hid1.shape)
    # other hidden layers here
    self.W_hid2 = xavier(n_hidden2 * n_hidden1).reshape(n_hidden2, n_hidden1) # first hidden layer of MLP
    self.b_hid2 = xavier(n_hidden2)
    self.rW_hid2 = np.zeros(self.W_hid2.shape)
    self.rb_hid2 = np.zeros(self.b_hid2.shape)
    self.W_out = xavier(n_classes * n_hidden2).reshape(n_classes, n_hidden2) # output layer
    self.b_out = xavier(n_classes)
    self.rW_out = np.zeros(self.W_out.shape)
    self.rb_out = np.zeros(self.b_out.shape)
    self.val_hist = []
    self.train_hist = []
  def train(self, X_train, Y_train, X_val, Y_val, s_epoch=0, s_batch=0):
    # train and val data are already shuffled.
    N = Y_train.shape[0]
    n_batches = N // batch_size
    extra = N % batch_size

    for self.epoch in range(s_epoch,max_epochs):
      self.c_matrix = np.zeros((n_classes, n_classes))  # confusion matrix for training
      train_loss = 0
      for self.batch in range(s_batch,n_batches):
        train_loss_batch = 0
        if self.batch < n_batches - 1:
          net_batch_size = batch_size
          batch_X = X_train[self.batch * batch_size:(self.batch+1) * batch_size]
          batch_Y = Y_train[self.batch * batch_size:(self.batch+1) * batch_size]
        else:
          net_batch_size = batch_size + extra
          batch_X = X_train[self.batch * batch_size:(self.batch+1) * batch_size + extra]
          batch_Y = Y_train[self.batch * batch_size:(self.batch+1) * batch_size + extra]
        # train model
        dW_in = np.zeros(self.W_in.shape)
        dW_out = np.zeros(self.W_out.shape)
        dW_rec = np.zeros(self.W_rec.shape)
        dW_hid1 = np.zeros(self.W_hid1.shape)
        dW_hid2 = np.zeros(self.W_hid2.shape)
        db_in = np.zeros(self.b_in.shape)
        db_out = np.zeros(self.b_out.shape)
        db_rec = np.zeros(self.b_rec.shape) 
        db_hid1 = np.zeros(self.b_hid1.shape) 
        db_hid2 = np.zeros(self.b_hid2.shape) 
        
        dW_in_tmp = np.zeros(self.W_in.shape)
        dW_out_tmp = np.zeros(self.W_out.shape)
        dW_rec_tmp = np.zeros(self.W_rec.shape)
        dW_hid1_tmp = np.zeros(self.W_hid1.shape)
        dW_hid2_tmp = np.zeros(self.W_hid2.shape)
        db_in_tmp = np.zeros(self.b_in.shape)
        db_out_tmp = np.zeros(self.b_out.shape)
        db_rec_tmp = np.zeros(self.b_rec.shape)
        db_hid1_tmp = np.zeros(self.b_hid1.shape)
        db_hid2_tmp = np.zeros(self.b_hid2.shape)

        dW_in_t = np.zeros(self.W_in.shape)
        dW_rec_t = np.zeros(self.W_rec.shape)
        db_in_t = np.zeros(self.b_in.shape)
        db_rec_t = np.zeros(self.b_rec.shape)
        N = batch_X.shape[0]
        # forward pass
        layers = []
        prev_v = np.zeros((N, n_units))
        for t in range(in_shape[0]):
          new_input = batch_X[:,t,:]  # N,3
          # print(new_input.shape)
          mul_in = np.dot(new_input, self.W_in.T) + self.b_in
          # print(mul_in.shape)
          mul_rec = np.dot(prev_v, self.W_rec.T) + self.b_rec
          add = mul_rec + mul_in
          v = tanh(add)
          mul_hid1 = np.dot(v, self.W_hid1.T) + self.b_hid1
          out_hid1 = tanh(mul_hid1)
          mul_hid2 = np.dot(mul_hid1, self.W_hid2.T) + self.b_hid2
          out_hid2 = tanh(mul_hid2)
          mul_out = np.dot(out_hid2, self.W_out.T) + self.b_out
          pred = softmax(mul_out)
          layers.append({'v':v, 'prev_v':prev_v, 'out_hid1':out_hid1, 'out_hid2':out_hid2, 'add':add})
          prev_v = v       

        train_loss_batch = - np.sum(batch_Y * np.log2(pred)) / net_batch_size
        train_loss += train_loss_batch
        for sample in range(pred.shape[0]):
          self.c_matrix[np.argmax(pred[sample]), np.argmax(batch_Y[sample])] += 1

        # minimize categorical cross entropy
        # derivative of softmax
        dpred = (pred - batch_Y)
        
        # backward pass
        for t in range(in_shape[0]):
          dW_out_tmp = np.dot(dpred.T, layers[t]['out_hid2'])
          db_out_tmp = np.dot(dpred.T, np.ones((N)))
          err_out = np.dot(dpred, self.W_out)
          delta_out = tanh_derivative(mul_hid2) * err_out

          dW_hid2_tmp = np.dot(delta_out.T, layers[t]['out_hid1'])
          db_hid2_tmp = np.dot(delta_out.T, np.ones((N)))
          err_hid2 = np.dot(delta_out, self.W_hid2)
          delta_hid2 = tanh_derivative(mul_hid1) * err_hid2

          dW_hid1_tmp = np.dot(delta_hid2.T, layers[t]['v'])
          db_hid1_tmp = np.dot(delta_hid2.T, np.ones((N)))
          err_hid1 = np.dot(delta_hid2, self.W_hid1)
          delta_hid1 = tanh_derivative(add) * err_hid1

          # dmul_rec = delta_hid * np.ones_like(mul_rec)
          # dprev_v = np.dot(np.transpose(self.W_rec), dmul_rec)
          delta = delta_hid1
          for i in range(t-1, max(-1, t-bptt_truncate-1), -1):
            # error = err_hid1 + dprev_v
            # delta = tanh_derivative(add) * error
            new_input = batch_X[:,t,:]
            dW_in_t = np.dot(delta.T, new_input)
            db_in_t = np.dot(delta.T, np.ones((N)))

            dW_rec_t = np.dot(delta.T, layers[t]['prev_v'])
            dB_rec_t = np.dot(delta.T, np.ones((N)))

            err_rec = np.dot(delta, self.W_rec)
            delta = tanh_derivative(layers[t]['add']) * err_rec

            dW_in_tmp += dW_in_t
            dW_rec_tmp += dW_rec_t
            db_in_tmp += db_in_t
            db_rec_tmp += db_rec_t
          dW_out += dW_out_tmp
          dW_hid2 += dW_hid2_tmp
          dW_hid1 += dW_hid1_tmp
          dW_in += dW_in_tmp
          dW_rec += dW_rec_tmp
          db_out += db_out_tmp
          db_hid2 += db_hid2_tmp
          db_hid1 += db_hid1_tmp
          db_in += db_in_tmp
          db_rec += db_in_tmp
          
        # update momentum terms
        dW_in, dW_rec, dW_hid1, dW_hid2, dW_out, db_in, db_rec, db_hid1, db_hid2, db_out = clip(dW_in, dW_rec, dW_hid1, dW_hid2, dW_out, db_in, db_rec, db_hid1, db_hid2, db_out)

        self.rW_in = momentum * self.rW_in + (1 - momentum) * (dW_in / net_batch_size) ** 2
        self.rW_out = momentum * self.rW_out + (1 - momentum) * (dW_out / net_batch_size) ** 2
        self.rW_rec = momentum * self.rW_rec + (1 - momentum) * (dW_rec / net_batch_size) ** 2
        self.rW_hid1 = momentum * self.rW_hid1 + (1 - momentum) * (dW_hid1 / net_batch_size) ** 2
        self.rW_hid2 = momentum * self.rW_hid2 + (1 - momentum) * (dW_hid2 / net_batch_size) ** 2
        self.rb_in = momentum * self.rb_in + (1 - momentum) * (db_in / net_batch_size) ** 2
        self.rb_out = momentum * self.rb_out + (1 - momentum) * (db_out / net_batch_size) ** 2
        self.rb_rec = momentum * self.rb_rec + (1 - momentum) * (db_rec / net_batch_size) ** 2
        self.rb_hid1 = momentum * self.rb_hid1 + (1 - momentum) * (db_hid1 / net_batch_size) ** 2
        self.rb_hid2 = momentum * self.rb_hid2 + (1 - momentum) * (db_hid2 / net_batch_size) ** 2

        # update after each mini-batch
        self.W_in -= learning_rate * dW_in / net_batch_size / np.sqrt(0.0000000000000000001 + self.rW_in)
        self.W_out -= learning_rate * dW_out / net_batch_size / np.sqrt(0.0000000000000000001 + self.rW_out)
        self.W_rec -= learning_rate * dW_rec / net_batch_size / np.sqrt(0.0000000000000000001 + self.rW_rec)
        self.W_hid1 -= learning_rate * dW_hid1 / net_batch_size / np.sqrt(0.0000000000000000001 + self.rW_hid1)
        self.W_hid2 -= learning_rate * dW_hid2 / net_batch_size / np.sqrt(0.0000000000000000001 + self.rW_hid2)
        self.b_in -= learning_rate * db_in / net_batch_size / np.sqrt(0.0000000000000000001 + self.rb_in)
        self.b_out -= learning_rate * db_out / net_batch_size / np.sqrt(0.0000000000000000001 + self.rb_out)
        self.b_rec -= learning_rate * db_rec / net_batch_size / np.sqrt(0.0000000000000000001 + self.rb_rec)
        self.b_hid1 -= learning_rate * db_hid1 / net_batch_size / np.sqrt(0.0000000000000000001 + self.rb_hid1)
        self.b_hid2 -= learning_rate * db_hid2 / net_batch_size / np.sqrt(0.0000000000000000001 + self.rb_hid2)

        # print('Epoch ', self.epoch+1, ', batch ', self.batch+1, ' / ', n_batches, ' => ', 'train_loss=', train_loss_batch)
      train_loss = train_loss / (n_batches - s_batch)
      val_loss, _ = categorical_cross_entropy(self.W_in, self.W_rec, self.W_hid1, self.W_hid2, self.W_out, self.b_in, self.b_rec, self.b_hid1, self.b_hid2, self.b_out, X_val, Y_val)
      self.train_hist.append(train_loss)
      self.val_hist.append(val_loss)
      print('Epoch ', str(self.epoch+1), ' => train_loss=', train_loss, ', val_loss=', val_loss)
      # if val_loss < val_loss_threshold:  # ???
      #  return train_hist, val_hist, W_in, b_in, W_rec, b_rec, W_out, b_out
      
    return self.train_hist, self.val_hist

In [ ]:
model = RNN()

In [ ]:
model.rW_out.shape

(6, 100)

In [ ]:
mul = model.train(X_train, Y_train, X_val, Y_val, s_batch=0, s_epoch=0)

Epoch  1  => train_loss= 7.153472167541775 , val_loss= 8.037330531658023
Epoch  2  => train_loss= 9.77278636617864 , val_loss= 9.7389848808166
Epoch  3  => train_loss= 11.27577290844025 , val_loss= 5.168801362059048
Epoch  4  => train_loss= 6.77520658364905 , val_loss= 5.339282971683203
Epoch  5  => train_loss= 6.044216402431624 , val_loss= 3.2143601225850262
Epoch  6  => train_loss= 6.347235954446859 , val_loss= 5.150225938396729
Epoch  7  => train_loss= 6.136491876757911 , val_loss= 4.279493828069977
Epoch  8  => train_loss= 6.150002241181289 , val_loss= 6.4219614948854975
Epoch  9  => train_loss= 6.057261230986258 , val_loss= 5.516124049178367
Epoch  10  => train_loss= 6.056868084827833 , val_loss= 5.142353638339502
